In [13]:
import pandas as pd
import numpy as np
import icecream as ic

'''
reshape 함수 종류

 - (1) pivot()
 90도 회전(rotation)
 컬럼 -> 인덱스로 전환

 - (2) stack(), unstack()
 stack()이 위에서 아래로 쌓아올린 구조라면
 unstack()은 순서 없이 놓여져 있다.

 - (3) melt() // 1~ 3번까지가 제일 자주 사용됨.

 - (4) wide_to_long()

 - (5) pd.crosstab()


'''

'\nreshape 함수 종류\n\n - (1) pivot(), pd.pivot_table()\n\n - (2) stack(), unstack()\n\n - (3) melt() // 1~ 3번까지가 제일 자주 사용됨.\n\n - (4) wide_to_long()\n\n - (5) pd.crosstab()\n\n\n'

In [23]:

# 30 다음 객체를 customer_id 를 인덱스로하고 product_code 를 컬럼으로, purchare_amount 를 값으로, 재구성하시오
df = pd.DataFrame({"customer_id":['kim','lee','park','song','yoon','kang','tak','ryu','jang'],
               "product_code":['com','phone','tv','com','phone','tv','com','phone','tv'],
               "grade":['A','A','A','A','A','A','B','B','B'],
               "purchase_amount":[30,10,0,40,15,30,0,0,10]})
print(df)
df = df.pivot(index='customer_id', columns='product_code', values='purchase_amount')
print(df)

  customer_id product_code grade  purchase_amount
0         kim          com     A               30
1         lee        phone     A               10
2        park           tv     A                0
3        song          com     A               40
4        yoon        phone     A               15
5        kang           tv     A               30
6         tak          com     B                0
7         ryu        phone     B                0
8        jang           tv     B               10
product_code   com  phone    tv
customer_id                    
jang           NaN    NaN  10.0
kang           NaN    NaN  30.0
kim           30.0    NaN   NaN
lee            NaN   10.0   NaN
park           NaN    NaN   0.0
ryu            NaN    0.0   NaN
song          40.0    NaN   NaN
tak            0.0    NaN   NaN
yoon           NaN   15.0   NaN


In [38]:
# 31 다음 객체를 customer_id, grade 를 인덱스로하고 product_code 를 컬럼으로, purchare_amount 를 값으로, 재구성하시오
df = pd.DataFrame({"customer_id":['kim','lee','park','song','yoon','kang','tak','ryu','jang'],
               "product_code":['com','phone','tv','com','phone','tv','com','phone','tv'],
               "grade":['A','A','A','A','A','A','B','B','B'],
               "purchase_amount":[30,10,0,40,15,30,0,0,10]})

df = df.pivot(index=['customer_id', 'grade'], columns ='product_code', values='purchase_amount')
df

,product_code,com,phone,tv
customer_id,grade,,,
jang,B,NaN,NaN,10.0
kang,A,NaN,NaN,30.0
kim,A,30.0,NaN,NaN
lee,A,NaN,10.0,NaN
park,A,NaN,NaN,0.0
ryu,B,NaN,0.0,NaN
song,A,40.0,NaN,NaN
tak,B,0.0,NaN,NaN
yoon,A,NaN,15.0,NaN


In [113]:
#32

df = pd.DataFrame(data=np.arange(16).reshape(4,4),
                  index=pd.MultiIndex.from_tuples([('kim','2019'),('kim','2020'),('lee', '2019'),('lee','2020')]),
                  columns=['com','phone','tv','notebook'])
print(df)
# print(type(df.stack())) pandas.core.series.Series
# 칼럼의 level은 1개 밖에 없으므로 stack(level=-1) 을 별도로 명기하지 않아도 됩니다.
df = df.stack(level=-1, dropna= True)# <class 'pandas.core.series.Series'>
print(df)

df = df.unstack(level=0)
print(df)

'''
결측값이 있는 데이터셋을 stack() 할 때 결측값을 제거할지(dropna=True),
아니면 결측값을 NaN으로 유지할지(dropna=False) 설정
unstack(level=-1), unstack(level=0), unstack(level=1) 별로
어떤 level이 칼럼으로 이동해서 unstack() 되는지 체크
'''

df = df.reset_index()
df.rename(columns={'level_0':'year','level_1':'product_code'},
          inplace=True)
df



          com  phone  tv  notebook
kim 2019    0      1   2         3
    2020    4      5   6         7
lee 2019    8      9  10        11
    2020   12     13  14        15
kim  2019  com          0
           phone        1
           tv           2
           notebook     3
     2020  com          4
           phone        5
           tv           6
           notebook     7
lee  2019  com          8
           phone        9
           tv          10
           notebook    11
     2020  com         12
           phone       13
           tv          14
           notebook    15
dtype: int32
               kim  lee
2019 com         0    8
     phone       1    9
     tv          2   10
     notebook    3   11
2020 com         4   12
     phone       5   13
     tv          6   14
     notebook    7   15


,year,product_code,kim,lee
0,2019,com,0,8
1,2019,phone,1,9
2,2019,tv,2,10
3,2019,notebook,3,11
4,2020,com,4,12
5,2020,phone,5,13
6,2020,tv,6,14
7,2020,notebook,7,15


In [117]:
'''
melt() 는 ID 변수를 기준으로 원래 데이터셋에 있던 여러개의 칼럼 이름을
'variable' 칼럼에 위에서 아래로 길게 쌓아놓고,
'value' 칼럼에 ID와 variable에 해당하는 값을
넣어주는 식으로 데이터를 재구조화
'''
df = pd.DataFrame({"customer_id":['kim','kim','lee','lee'],
               "product_code":['com','phone','tv','tab'],
               "purchase_count":[1,2,3,4],
               "purchase_amount":[100,200,300,400]})
df

# (1) pd.melt(data, id_vars=['id1', 'id2', ...]) 를 사용한 데이터 재구조화
df = df.melt(id_vars=['customer_id','product_code'],
             var_name='vaiables', value_name='value')
df

# (2) pd.melt() 의 variable 이름, value 이름 부여하기 : var_name, value_name


,customer_id,vaiables,value
0,kim,product_code,com
1,kim,product_code,phone
2,lee,product_code,tv
3,lee,product_code,tab
4,kim,purchase_count,1
5,kim,purchase_count,2
6,lee,purchase_count,3
7,lee,purchase_count,4
8,kim,purchase_amount,100
9,kim,purchase_amount,200


In [150]:
np.random.seed(10)
df = pd.DataFrame({"Class1":{0:"A",1:"B",2:"C"},
                  "Class2":{0:"D",1:"E",2:"F"},
                  "Score1":{0:"2.5",1:"1.2",2:"0.7"},
                  "Score2":{0:"3.2",1:"1.3",2:"0.1"},
                   "value": dict(zip(range(3),np.random.randn(3)))})
df["seq"] = df.index
df
print(df)
'''
pd.wide_to_long() 함수를 써서 데이터를 재구조화
wide_to_long()은 pivot() 이나 stack() 과는 다르게
"칼럼 이름의 앞부분"과 나머지 "칼럼 이름의 뒷부분"을 구분해서,
칼럼 이름의 앞부분을 칼럼 이름으로,
칼럼 이름의 나머지 뒷부분을 행(row)의 원소로 해서
세로로 길게(long~) 쌓아 줍니다.
pd.widt_to_long() 함수를 한번 사용해서
가로로 넓은 데이터(wide~)를 세로로 길게(long~) 재구조화 하시오.
pd.wide_to_long(data, ["col_prefix_1", "col_prefix_2"], i="idx_1", j="idx_2")
'''
df = pd.wide_to_long(df,['Class','Score'], i="seq", j="index")
df


# df = df.wide_to_long(['Class','Score'],i="index", j="value")


  Class1 Class2 Score1 Score2     value  seq
0      A      D    2.5    3.2  1.331587    0
1      B      E    1.2    1.3  0.715279    1
2      C      F    0.7    0.1 -1.545400    2


,,value,Class,Score
seq,index,,,
0,1,1.331587,A,2.5
1,1,0.715279,B,1.2
2,1,-1.545400,C,0.7
0,2,1.331587,D,3.2
1,2,0.715279,E,1.3
2,2,-1.545400,F,0.1


  Class1 Class2 Score1 Score2     value  seq
0      A      D    2.5    3.2  1.331587    0
1      B      E    1.2    1.3  0.715279    1
2      C      F    0.7    0.1 -1.545400    2


,,value,Class,Score
seq,index,,,
0,1,1.331587,A,2.5
1,1,0.715279,B,1.2
2,1,-1.545400,C,0.7
0,2,1.331587,D,3.2
1,2,0.715279,E,1.3
2,2,-1.545400,F,0.1


  Class1 Class2 Score1 Score2     value  seq
0      A      D    2.5    3.2  1.331587    0
1      B      E    1.2    1.3  0.715279    1
2      C      F    0.7    0.1 -1.545400    2


...

,,value,Class,Score
seq,index,,,
0,1,1.331587,A,2.5
1,1,0.715279,B,1.2
2,1,-1.545400,C,0.7
0,2,1.331587,D,3.2
1,2,0.715279,E,1.3
2,2,-1.545400,F,0.1


  Class1 Class2 Score1 Score2     value  seq
0      A      D    2.5    3.2  1.331587    0
1      B      E    1.2    1.3  0.715279    1
2      C      F    0.7    0.1 -1.545400    2


,,value,Class,Score
seq,index,,,
0,1,1.331587,A,2.5
1,1,0.715279,B,1.2
2,1,-1.545400,C,0.7
0,2,1.331587,D,3.2
1,2,0.715279,E,1.3
2,2,-1.545400,F,0.1


  Class1 Class2 Score1 Score2     value  seq
0      A      D    2.5    3.2  1.331587    0
1      B      E    1.2    1.3  0.715279    1
2      C      F    0.7    0.1 -1.545400    2


,,value,Class,Score
seq,index,,,
0,1,1.331587,A,2.5
1,1,0.715279,B,1.2
2,1,-1.545400,C,0.7
0,2,1.331587,D,3.2
1,2,0.715279,E,1.3
2,2,-1.545400,F,0.1


In [102]:
'''
pd.crosstab()
범주형 변수로 되어있는 요인(factors)별로 교차분석(cross tabulations) 해서,
행, 열 요인 기준 별로 빈도를 세어서 도수분포표(frequency table),
교차표(contingency table) 를 생성한다

# 범주형 변수란 고유한 값이나 범주 수가 제한된 변수(예: 성별 또는 종교)이다.
# 교차분석(cross-tabulation analysis)은 '범주형'으로 구성된 자료들 간의 연관관계를 확인하기 위해
   교차표를 만들어 관계를 확인하는 분석 방법이다.
'''
df = pd.DataFrame({"학생 ID" :["kim","kim","kim","lee","lee","park"],
                   "1학기 학점":["A","A","A","B","B","B"],
                   "2학기 학점":["D","D","D","C","C","D"]})
print(type(df))
df
# (1) 교차표(contingency table, frequency table) 만들기 : pd.crosstab(index, columns)
df2 = pd .crosstab(index=df['1학기 학점'], columns=df["2학기 학점"])
df2

# (2) Multi-index, Multi-level로 교차표 만들기 : pd.crosstab([id1, id2], [col1, col2])
df3 = pd.crosstab(index=df['학생 ID'], columns=[df["1학기 학점"],df["2학기 학점"]])
df3

# (3) 교차표의 행 이름, 열 이름 부여 : pd.crosstab(rownames=['xx'], colnames=['aa'])
df4 = pd.crosstab(index=df['학생 ID'], columns=[df["1학기 학점"],df["2학기 학점"]],
                 rownames=["학점"], colnames=['ID','ID'])
df4

# (4) 교차표의 행 합, 열 합 추가하기 : pd.crosstab(margins=True)
df5 = pd.crosstab(index=df['학생 ID'], columns=[df["1학기 학점"],df["2학기 학점"]],
                  margins=True)
df5

# (5) 구성비율로 교차표 만들기 : pd.crosstab(normalize=True)
df6 = pd.crosstab(index=df['학생 ID'], columns=[df["1학기 학점"],df["2학기 학점"]],
                  normalize=True)
df6


<class 'pandas.core.frame.DataFrame'>


1학기 학점    A         B          
2학기 학점    D         C         D
학생 ID                          
kim     0.5  0.000000  0.000000
lee     0.0  0.333333  0.000000
park    0.0  0.000000  0.166667